<a href="https://colab.research.google.com/github/once-upon-an-april/Thuc-Hanh-Deep-Learning-trong-Khoa-Hoc-Du-Lieu-DS201.Q11.1/blob/main/Bai2/22520975_Lab3_Bai2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==============================================================================
# BÀI 2: PHÂN LOẠI CẢM XÚC TRÊN DATASET UIT-VSFC SỬ DỤNG GRU
# (5 Lớp, Hidden Size 256, Adam Optimizer, F1 Score)
# ==============================================================================
!pip install pyvi

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import numpy as np
import pandas as pd
import re
import string
from collections import Counter
from sklearn.metrics import f1_score, classification_report
from datasets import load_dataset
from pyvi import ViTokenizer

# Kiểm tra thiết bị (GPU/CPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
# ==============================================================================
# PHẦN 1: TẢI VÀ TIỀN XỬ LÝ DỮ LIỆU (SỬ DỤNG HUGGING FACE DATASETS)
# ==============================================================================

print("\n>>> 1. Đang tải dataset từ HuggingFace...")

# Tải dataset từ HuggingFace (ưu tiên bản Parquet để tránh lỗi script bảo mật)
try:
    ds = load_dataset("uitnlp/vietnamese_students_feedback", name="default", revision="refs/convert/parquet")
except Exception:
    # Fallback: Dùng trust_remote_code nếu không tải được bản parquet
    ds = load_dataset("uitnlp/vietnamese_students_feedback", trust_remote_code=True)

# Chuyển đổi sang Pandas DataFrame để xử lý tiếp
train_df = pd.DataFrame(ds['train'])
val_df = pd.DataFrame(ds['validation'])
test_df = pd.DataFrame(ds['test'])

print(f"Dataset Loaded. Train: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# Hàm tiền xử lý
def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(f'[{re.escape(string.punctuation)}]', '', text)
    text = ViTokenizer.tokenize(text) # Tách từ tiếng Việt
    return text

print(">>> Preprocessing (Clean & Tokenize)...")
train_df['sentence_cleaned'] = train_df['sentence'].apply(preprocess_text)
val_df['sentence_cleaned'] = val_df['sentence'].apply(preprocess_text)
test_df['sentence_cleaned'] = test_df['sentence'].apply(preprocess_text)

In [ ]:
# ==============================================================================
# PHẦN 2: XÂY DỰNG TỪ ĐIỂN & DATASET
# ==============================================================================

# Cấu hình Hyperparameters
MAX_LEN = 50
BATCH_SIZE = 64
EMBEDDING_DIM = 100
HIDDEN_SIZE = 256  # Yêu cầu đề bài
NUM_LAYERS = 5     # Yêu cầu đề bài
NUM_CLASSES = 3
LEARNING_RATE = 0.001
EPOCHS = 100

print("\n>>> 2. Xây dựng Vocabulary...")
all_words = " ".join(train_df['sentence_cleaned']).split()
word_counts = Counter(all_words)
vocab = sorted([w for w, c in word_counts.items() if c >= 2])

word2idx = {w: i+2 for i, w in enumerate(vocab)}
word2idx['<PAD>'] = 0
word2idx['<UNK>'] = 1

print(f"Vocab Size: {len(word2idx)}")

def vectorize(text, word2idx, max_len):
    words = text.split()
    idxs = [word2idx.get(w, word2idx['<UNK>']) for w in words]
    if len(idxs) < max_len:
        idxs = idxs + [word2idx['<PAD>']] * (max_len - len(idxs))
    else:
        idxs = idxs[:max_len]
    return idxs

class VSFC_Dataset(Dataset):
    def __init__(self, df, word2idx, max_len):
        self.sentences = df['sentence_cleaned'].values
        self.labels = df['sentiment'].values
        self.word2idx = word2idx
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        vec = vectorize(self.sentences[idx], self.word2idx, self.max_len)
        label = self.labels[idx]
        return torch.tensor(vec, dtype=torch.long), torch.tensor(label, dtype=torch.long)

train_loader = DataLoader(VSFC_Dataset(train_df, word2idx, MAX_LEN), batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(VSFC_Dataset(val_df, word2idx, MAX_LEN), batch_size=BATCH_SIZE)
test_loader = DataLoader(VSFC_Dataset(test_df, word2idx, MAX_LEN), batch_size=BATCH_SIZE)

In [ ]:
# ==============================================================================
# PHẦN 3: XÂY DỰNG MODEL GRU
# ==============================================================================

class SentimentGRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, output_dim, n_layers):
        super(SentimentGRU, self).__init__()

        # 1. Embedding
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)

        # 2. GRU Layer (Thay thế LSTM bằng GRU)
        # GRU không có cell state, chỉ có hidden state
        self.gru = nn.GRU(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=n_layers,
            batch_first=True,
            dropout=0.3
        )

        # 3. Output Layer
        self.fc = nn.Linear(hidden_size, output_dim)

    def forward(self, text):
        # text: [batch, seq_len]
        embedded = self.embedding(text)

        # GRU output trả về: output, hidden
        # (Khác với LSTM trả về: output, (hidden, cell))
        output, hidden = self.gru(embedded)

        # hidden shape: [num_layers, batch, hidden_size]
        # Lấy hidden state của lớp cuối cùng (layer n)
        last_hidden = hidden[-1, :, :]

        return self.fc(last_hidden)

# Khởi tạo mô hình
model = SentimentGRU(len(word2idx), EMBEDDING_DIM, HIDDEN_SIZE, NUM_CLASSES, NUM_LAYERS).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

print("\n>>> 3. Kiến trúc mô hình GRU:")
print(model)

In [ ]:
# ==============================================================================
# PHẦN 4: HUẤN LUYỆN VÀ ĐÁNH GIÁ
# ==============================================================================

def compute_f1(loader, model):
    model.eval()
    all_preds, all_labels = [], []
    with torch.no_grad():
        for text, labels in loader:
            text, labels = text.to(device), labels.to(device)
            preds = model(text).argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    return f1_score(all_labels, all_preds, average='weighted')

print("\n>>> 4. Bắt đầu huấn luyện...")
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0

    for text, labels in train_loader:
        text, labels = text.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(text)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    val_f1 = compute_f1(val_loader, model)
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {total_loss/len(train_loader):.4f} | Val F1: {val_f1:.4f}")

In [ ]:
# ==============================================================================
# PHẦN 5: KẾT QUẢ CUỐI CÙNG TRÊN TẬP TEST
# ==============================================================================
print("\n>>> 5. Kết quả trên tập TEST:")
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for text, labels in test_loader:
        text, labels = text.to(device), labels.to(device)
        preds = model(text).argmax(dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print(classification_report(all_labels, all_preds, target_names=['Negative', 'Neutral', 'Positive'], digits=4))